# 2D Image to 3D Model

## Loading Material Properties From json

In [36]:
import json
from PIL import Image
import numpy as np
import trimesh

with open('resized_img_processed_model_mapping.json', 'r') as f:
    img_to_mod_map = json.load(f)

with open('material_properties.json', 'r') as f:
    material_properties = json.load(f)


def load_preprocess_img(img_path):
    
    img = Image.open(img_path)
    img_array = np.array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def simplify_mesh(mesh, target_vertices=1024):
    simplified_mesh = mesh.simplify_quadratic_decimation(target_vertices)
    return simplified_mesh

def upsample_mesh(mesh, target_vertices=1024):
    sampled_points, _ = trimesh.sample.sample_surface_even(mesh, target_vertices)
    return sampled_points


def load_3d_model(model_path, target_vertices=1024):
    mesh = trimesh.load(model_path)

    if len(mesh.vertices) > target_vertices:
        simplified_mesh = simplify_mesh(mesh, target_vertices)
        return simplified_mesh

    elif len(mesh.vertices) < target_vertices:
        upsampled_mesh = upsample_mesh(mesh, target_vertices)
        return upsampled_mesh
    
    return mesh


def get_material_prop(img_path, img_to_mod_map, material_properties):
    model_path = img_to_mod_map.get(img_path, None)

    if model_path is None:
        raise ValueError(f"No model found for img: {img_path}")

    material_path = model_path.replace('simple_normal_model.obj', 'model.mtl')
    material_path = material_path.replace('../model/', '')
    materials = material_properties.get(material_path, None)
    print(material_path)
    if materials is None:
        print(f"No materials found for model: {material_path}. Using default materials")
        materials = {
            'Kd': [1.0, 1.0, 1.0],  # Default white diffuse color
            'Ks': [0.0, 0.0, 0.0],  # No specular highlights
            'Ns': 96.078431,        # Default shininess
            'd': 1.0                # Full opacity
        }
    return materials


def normalize_materials(material):
    max_shine = 1000

    normalized_material = {
        'Kd': material.get('Kd', [1.0, 1.0, 1.0]),
        'Ks': material.get('Ks', [0.0, 0.0, 0.0]),
        'Ns': material.get('Ns', 96.078431) / max_shine,
        'd': material.get('d', 1.0)
    }

    return normalized_material


def preprocess_image_with_material(img_path, img_to_mod_map, material_properties):
    img = load_preprocess_img(img_path)

    model_path = img_to_mod_map.get(img_path)
    mesh = load_3d_model(model_path, target_vertices=1024)

    materials = get_material_prop(img_path, img_to_mod_map, material_properties)
    normalized_materials = normalize_materials(materials)

    return img, mesh, normalized_materials

## Data Generator

In [40]:
from tensorflow.keras.utils import Sequence
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataGenerator(Sequence):
    def __init__(self, img_paths, img_to_mod_map, material_properties, batch_size=32, dim=(256, 256, 3), augment=False):
        self.img_paths = img_paths
        self.img_to_mod_map = img_to_mod_map
        self.material_properties = material_properties
        self.batch_size = batch_size
        self.dim = dim
        self.augment = augment

        # Image data augmentation
        self.image_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            brightness_range=[0.8, 1.2],
            fill_mode='nearest'
        )

    def __len__(self):
        return int(np.floor(len(self.img_paths) / self.batch_size))

    def __getitem__(self, index):
        batch_img_paths = self.img_paths[index * self.batch_size:(index + 1) * self.batch_size]
        imgs = []
        materials = []
        meshes = []

        for img_path in batch_img_paths:
            img = load_preprocess_img(img_path)
            img = np.squeeze(img, axis=0)

            if self.augment:
                img = self.image_datagen.random_transform(img)
            
            img = np.expand_dims(img, axis=0)
            imgs.append(img)

            material = get_material_prop(img_path, self.img_to_mod_map, self.material_properties)
            normalized_material = normalize_materials(material)
            materials.append(normalized_material)

            model_path = self.img_to_mod_map.get(img_path)
            mesh = load_3d_model(model_path)
            
            if hasattr(mesh, 'vertices'):
                meshes.append(mesh.vertices)
            else:
                print(f"Warning: No vertices found in model: {model_path}. Skipping.")
                meshes.append(np.zeros((1024, 3)))

        imgs = np.vstack(imgs)
        materials = np.array(materials)
        meshes = np.array([mesh.vertices for mesh in meshes])

        return [imgs, materials], meshes

    def on_epoch_end(self):
        np.random.shuffle(self.img_paths)


## Model Architecture

In [41]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2

image_input = Input(shape=(256, 256, 3), name='image_input')
resnet_base = ResNet50V2(weights='imagenet', include_top=False, input_tensor=image_input)

x = resnet_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)

material_input = Input(shape=(4,), name='material_input')
material_dense = Dense(64, activation='relu')(material_input)
material_dense = Dense(64, activation='relu')(material_dense)

combined = Concatenate()([x, material_dense])

z = Dense(256, activation='relu')(combined)
z = Dense(512, activation='relu')(z)

num_vertices = 1024
output = Dense(num_vertices * 3, activation='linear', name='output')(z)

model = Model(inputs=[image_input, material_input], outputs=output)


# Training

In [42]:
from sklearn.model_selection import train_test_split
import os

img_dir = "../resized_images/"
img_paths = []
for root, dirs, files in os.walk(img_dir):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            img_paths.append(os.path.join(root, file))

print(f"Found {len(img_paths)} images.")

train_img_paths, temp_img_paths = train_test_split(img_paths, test_size=0.3, random_state=42)
val_img_paths, test_img_paths = train_test_split(temp_img_paths, test_size=0.5, random_state=42)

train_data_gen = DataGenerator(train_img_paths, img_to_mod_map, material_properties, batch_size=32, augment=True)
val_data_gen = DataGenerator(val_img_paths, img_to_mod_map, material_properties, batch_size=32, augment=False)
test_data_gen = DataGenerator(test_img_paths, img_to_mod_map, material_properties, batch_size=32, augment=False)

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(
    train_data_gen,
    epochs=20,
    validation_data=val_data_gen
)

Found 10000 images.
table/IKEA_DOCKSTA/model.mtl


only got 823/1024 samples!


chair/IKEA_INGOLF/model.mtl


only got 989/1024 samples!


sofa/IKEA_MANSTAD/model.mtl
desk/IKEA_LIATORP/model.mtl


...

only got 899/1024 samples!


bed/IKEA_NORDLI/model.mtl


only got 794/1024 samples!


desk/IKEA_BESTA_1/model.mtl


AttributeError: 'numpy.ndarray' object has no attribute 'vertices'

## Plot Data

In [ ]:
import mayplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()